# Inference Calculation

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
cd ../TileEM/

/Users/dorislee/Desktop/Research/seg/crowd-seg/analysis/TileEM


In [4]:
import shapely
import pandas as pd
from tqdm import tqdm
import pickle as pkl
from analysis_toolbox import *

In [47]:
DATA_DIR="final_all_tiles"

------------------------------

# Posterior 

In [1]:
cd ..

/Users/dorislee/Desktop/Research/seg/crowd-seg/analysis


In [2]:
# worker_obj_tbl=pd.read_csv("worker_obj_qualities.csv")

In [3]:
# objid =1 
# wid=75
# float(Qj[(Qj["object_id"]==objid)&(Qj["worker_id"]==wid)]["Qj"])

In [32]:
def compute_PR(objid,solnset,tiles):
    '''
    Compute precision recall against ground truth bounding box
    for a given solution set and tile coordinates.
    '''
    if len(solnset)==1:
        joined_bb=tiles[solnset]
        problematic_tiles=[]
    else:
        try:
            joined_bb,problematic_tiles = join_tiles(solnset,tiles)
        except(ValueError):
            return -1,-1
    ground_truth_match = my_BBG[my_BBG.object_id==objid]
    x_locs,y_locs =  process_raw_locs([ground_truth_match["x_locs"].iloc[0],ground_truth_match["y_locs"].iloc[0]])
    BBG = shapely.geometry.Polygon(zip(x_locs,y_locs))
    if problematic_tiles!=[]:
        intersect_area =0
        joined_bb_area =0
        for jbb in joined_bb:
            ia = intersection_area(BBG,jbb)
            intersect_area += ia
            joined_bb_area += jbb.area
    else:
        intersect_area=intersection_area(BBG,joined_bb)
        joined_bb_area =joined_bb.area
    if float(joined_bb_area)!=0:
        precision = intersect_area/float(joined_bb_area)
    else:
        # Empty solution set
        precision = -1
    recall = intersect_area/BBG.area
    #This patches up PR>1 because intersect_area using the slow intersection method can sometimes be larger since overlap area may not be completely deducted, so the estimate is a bit larger than the actual area, but not by much 
    if recall>1:
        recall=1
    if precision>1:
        precision=1
    return precision,recall

In [18]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
from TileEM_plot_toolbox import *
from TileEM_Models import *
gamma_properties=False
# DATA_DIR = "output"
topTilePickHeuristic="area"
DATA_DIR="final_all_tiles"
os.chdir(DATA_DIR)
try: 
    topTilePickHeuristic=sys.argv[1]
except(IndexError):
    topTilePickHeuristic="area"

topk = 40
training_tbl = []
my_BBG  = pd.read_csv("../../my_ground_truth.csv")
import itertools

In [34]:
gamma_properties=False
topTilePickHeuristic="area"

In [36]:
cd final_all_tiles/

/Users/dorislee/Desktop/Research/seg/crowd-seg/analysis/TileEM/final_all_tiles


In [46]:
for objid in tqdm(object_lst):
    # print "working on obj",objid
    #Get Tile information for that object
    worker_ids,worker_precision_lst,worker_recall_lst = compute_worker_PR_obj(objid,return_worker_id=True)
    tiles = pkl.load(open("vtiles{1}.pkl".format(DATA_DIR,objid),'r'))
    worker_ids = pkl.load(open("worker{1}.pkl".format(DATA_DIR,objid),'r'))
    indicatorMat = pkl.load(open("indMat{1}.pkl".format(DATA_DIR,objid),'r'))
    if gamma_properties : 
        gammas = pkl.load(open("gfile{}.pkl".format(objid),'r'))
        if  list(gammas[0])==[] and topTilePickHeuristic=='gamma' :
            print "No Gamma information for this object, going onto the next"
            continue

    #using the area information in the last row 
    tile_area = np.array(indicatorMat[-1])
    # Loop through all combinations of 20 randomly chosen tiles 
    # tile_subset_idx = np.random.choice(np.arange(len(tiles)),20)
    ground_truth_match = my_BBG[my_BBG.object_id==objid]
    x_locs,y_locs =  process_raw_locs([ground_truth_match["x_locs"].iloc[0],ground_truth_match["y_locs"].iloc[0]])
    BBG = shapely.geometry.Polygon(zip(x_locs,y_locs))
    tile_area_ratio = tile_area/BBG.area

    if topTilePickHeuristic=="area": 
        # Loop through all combinations of 40 top-area tiles 
        # print "area heuristic"
        tile_subset_idx = tile_area_ratio.argsort()[::-1][:topk]
    elif topTilePickHeuristic=="gamma":
        # print "gamma"
        tile_subset_idx = np.argsort(gammas[0])[::-1][:topk]
    elif topTilePickHeuristic=="majvote":
        # print "majvote"
        tile_votes = np.sum(indicatorMat[:-1],axis=0)
        tile_subset_idx = np.argsort(tile_votes)[::-1][:topk]

    rand_subset =[]
    for i in range(3000):
        NumTilesInCombo= np.random.randint(0,topk)
        tidxInCombo= np.random.choice(tile_subset_idx,NumTilesInCombo,replace=False)
        rand_subset.append(tidxInCombo)
    # Compute feature properties for T'
    for Tprime in rand_subset:
        p,r =compute_PR(objid,Tprime,tiles)
        gvals=[]
        experiment_idx=0
        # Majority Votes 
        region_votes=[]
        Tareas=[]
        for tidx in Tprime:
            #Number of votes for that tile
            region_votes.append(np.sum(indicatorMat[:-1][:,tidx]))

            if gamma_properties:
                if gammas!=[]:
                    gvals.append(gammas[experiment_idx][tidx])
                else:
                    gvals.append(0)

            Tareas.append(Polygon(tiles[tidx]).area)
        pTprime_val = pTprimeBasic(objid,Tprime,BBG)
        pTprimeLSA_val = pTprimeLSA(objid,Tprime,BBG,10)
        if pTprime_val==0: break
        training_tbl.append([objid,Tprime,np.sum(region_votes), np.mean(region_votes),np.sum(gvals),np.mean(gvals),\
                             np.sum(Tareas),np.mean(Tareas),pTprime_val,pTprimeLSA_val,p,r])

# df = pd.DataFrame(training_tbl,columns=["objid","T prime","Total Votes","Average Votes","Total gamma value","Average gamma value",\
#                                         "Total area","Average area","pTprime","Precision","Recall"])
# df.to_csv("tile_combo_metric_{}.csv".format(topTilePickHeuristic))

  0%|          | 0/47 [00:00<?, ?it/s]

[]
363
[]

ERROR:shapely.geos:TopologyException: side location conflict at 367.0273972602705 169.4657534245982



0
[]

ERROR:shapely.geos:TopologyException: found non-noded intersection between LINESTRING (316.737 150.053, 320.667 148.333) and LINESTRING (318.8 149.64, 322.261 147.217) at 320.66666666666669 148.33333333333331


...

ERROR:shapely.geos:TopologyException: found non-noded intersection between LINESTRING (353.812 196.491, 282.5 212.5) and LINESTRING (290 210.667, 282.5 212.5) at 282.50000000000011 212.49999999999997



312
[]

 19%|█▉        | 9/47 [03:38<15:22,  0.04it/s]


1248


KeyboardInterrupt: 

In [47]:
pd.DataFrame(training_tbl,columns=["objid","T prime","Total Votes","Average Votes","Total gamma value","Average gamma value",\
                                         "Total area","Average area","pTprime","pTprime[Athres>10]","Precision","Recall"])

,objid,T prime,Total Votes,Average Votes,Total gamma value,Average gamma value,Total area,Average area,pTprime,pTprime[Athres>10],Precision,Recall
0,1,"[3767, 3159, 2257, 2428, 2781, 3336, 2445, 367...",283,25.727273,0,NaN,18871.976834,1715.634258,1.095964e-108,1.222878e-105,0.018212,0.068151
1,1,"[285, 1612, 2583, 2781, 1755, 2531, 2257, 3675...",294,22.615385,0,NaN,23218.073707,1786.005670,2.253423e-127,1.701118e-122,0.190553,0.877514
2,1,"[3767, 174, 77, 3057, 2531, 371, 3161, 1755, 2...",597,22.111111,0,NaN,4986.041939,184.668220,5.760010e-265,3.625809e-260,0.897571,0.882221
3,1,"[3336, 2583, 2445, 371, 1755, 1824, 169, 2951,...",483,19.320000,0,NaN,19288.579292,771.543172,5.717877e-243,5.004380e-241,0.020636,0.078953
4,1,"[2951, 77, 169, 2534, 3159, 1974, 2531, 2428, ...",309,20.600000,0,NaN,432.627313,28.841821,4.315695e-147,1.412164e-141,0.279787,0.020740
5,1,"[2534, 370, 3479, 3681, 3675]",102,20.400000,0,NaN,123.667642,24.733528,4.111183e-52,1.646538e-51,0.497227,0.012210
6,1,"[2583, 3159, 3057, 3161, 1974, 2101, 3162, 338...",430,23.888889,0,NaN,19094.250951,1060.791719,3.177159e-177,2.114951e-170,0.016169,0.061243
7,1,"[3336, 3674, 3160, 1974, 3678, 2445, 3485, 370...",590,21.851852,0,NaN,5241.318183,194.122896,1.164873e-264,7.430687e-255,0.842359,0.863354
8,1,"[3767, 2101, 77, 285, 3160, 2783, 3162, 2540, ...",316,22.571429,0,NaN,656.518746,46.894196,5.505632e-135,2.469968e-128,0.035470,0.004456
9,1,"[1612, 285, 3674, 3681, 2257, 3162, 3479, 1755]",214,26.750000,0,NaN,4587.797708,573.474713,4.451520e-80,3.335171e-75,0.921157,0.839174
